In [6]:
import os, shutil
import fileinput as fi
import glob
import moviepy.editor as mp
import pandas as pd
import random
import time

from pydub import AudioSegment

from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from itertools import product


os.chdir(path=r"D:\02_batch_2")
print("Current Working Directory " , os.getcwd())

df_dirname = []
for folder in sorted(os.listdir(os.getcwd())):
    df_dirname.append((folder))

df_dirname = pd.DataFrame(df_dirname, columns=['Folder'])
df_dirname.sort_values(by='Folder',inplace=True)

Current Working Directory  D:\02_batch_2


### Arabic Annotation collection

In [2]:
li = []
path = r'D:/02_batch_2/'

for i,dname in enumerate(df_dirname['Folder']):

    files = glob.glob(path + dname + '/*.txt')
    #print(f'Successfully created dataframe for {dname}')
    for i,f in enumerate(files):

        parse_error = False
        file_not_found = False

        stock = os.path.basename(f)            
        temp_df = pd.read_csv(f, delimiter="\t", error_bad_lines=False)
        temp_df['filePath'] = path + dname + '/'
        temp_df['textFileName'] = stock     
        temp_df['audioFileName'] = temp_df['textFileName'].replace('.txt', '.wav', regex=True)
        temp_df['duration'] = temp_df.stop-temp_df.start
        li.append(temp_df)
      #  print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

df = pd.concat(li, axis=0)
print(df.shape)
df.head(2)

b'Skipping line 18: expected 5 fields, saw 7\n'
b'Skipping line 12: expected 5 fields, saw 6\n'
b'Skipping line 5: expected 5 fields, saw 8\n'


(507921, 9)


speaker gender      start       stop  \
0  speakeA      M   8.998978  23.522329   
1  speakeA      M  23.522329  25.772074   

                                          transcript  \
0  الحقوني الحقوني بطني بطني بموت  ما أقدر الحقون...   
1                             أضحك عليكو بس أختبركو.   

                           filePath               textFileName  \
0  D:/02_batch_2/37_degrees_series/  37_degrees_series_0-0.txt   
1  D:/02_batch_2/37_degrees_series/  37_degrees_series_0-0.txt   

               audioFileName   duration  
0  37_degrees_series_0-0.wav  14.523351  
1  37_degrees_series_0-0.wav   2.249745

In [ ]:
files = []
sumDuration = 0.0
n = random.randint(0,len(df)-1)

fname = str(df.iloc[n,5] + df.iloc[n,7])
sumDuration = sumDuration + df.iloc[n,8].astype('float32')

split_audio(fname,df.iloc[n,2],df.iloc[n,3],)

print(fname)

# AUXILLIARY CODE

### Video Clipping

In [ ]:
hours = 180000;
duration_sum = 0;
x=0;
dest_filepath = r'D:/nf/audio/'

df_selection = pd.DataFrame(
    columns=['row_no','src_filepath','src_filepath_txt','src_filepath_wav','dest_filepath','dest_filepath_txt','dest_filepath_wav','duration','start','end']
    )

while (hours>=duration_sum):
    
    n = random.randint(0,len(df))
    fname = str(df.iloc[n,5] + df.iloc[n,7])
    duration_sum = duration_sum + df.iloc[n,8].astype('float32')

    df_selection[x,0] = n
    df_selection[x,1] = str(df.iloc[n,5])
    df_selection[x,2] = str(df.iloc[n,6])
    df_selection[x,3] = str(df.iloc[n,7])
    df_selection[x,4] = dest_filepath
    df_selection[x,5] = str(x) + "_" + str(df.iloc[n,6])
    df_selection[x,6] = str(x) + "_" + str(df.iloc[n,7])
    df_selection[x,7] = df.iloc[n,8].astype('float32')
    df_selection[x,8] = df.iloc[n,2].astype('float32')
    df_selection[x,9] = df.iloc[n,3].astype('float32')

    x=x+1  
    start = time.time()
    my_clip = AudioSegment.from_wav(fname)  
    my_clip = new_audio[df.iloc[n,2]:df.iloc[n,3]]    
    my_clip
    
    #clip = my_clip.subclip(df.iloc[n,2],df.iloc[n,3])
    clip.write_audiofile(dest_filepath + str(x) + "_"+ str(df.iloc[n,7]) +".mp3")
    #time.sleep(1)
    end = time.time()    

    print(duration_sum)
    print(f"Runtime of the program is {end - start}")
    # clip.write_audiofile(dest_filepath + df_selection[x,6])


df_selection.head(4) 

In [7]:
def random_selection(outPath,hours):
    
    x = 0
    duration_sum = 0


    df_selection = pd.DataFrame(
        columns=['row_no','src_filepath','src_filepath_txt','src_filepath_wav','dest_filepath','dest_filepath_txt','dest_filepath_wav','duration','start','end']
    )

    while (hours>=duration_sum):
        
        n = random.randint(0,len(df))
        fname = str(df.iloc[n,5] + df.iloc[n,7])
        duration_sum = duration_sum + df.iloc[n,8].astype('float32')

        df_selection[x,0] = n
        df_selection[x,1] = str(df.iloc[n,5])
        df_selection[x,2] = str(df.iloc[n,6])
        df_selection[x,3] = str(df.iloc[n,7])
        df_selection[x,4] = outPath
        df_selection[x,5] = str(x) + "_" + str(df.iloc[n,6])
        df_selection[x,6] = str(x) + "_" + str(df.iloc[n,7])
        df_selection[x,7] = df.iloc[n,8].astype('float32')
        df_selection[x,8] = df.iloc[n,2].astype('float32')
        df_selection[x,9] = df.iloc[n,3].astype('float32')

        x=x+1  
        start = time.time()
        my_clip = mp.AudioFileClip(fname)    
        
        clip = my_clip.subclip(df.iloc[n,2],df.iloc[n,3])
        clip.write_audiofile(outPath + str(x) + "_"+ str(df.iloc[n,7]) +".mp3")
        #time.sleep(1)
        end = time.time()
        

        print(duration_sum)
        print(f"Runtime of the program is {end - start}")
        # clip.write_audiofile(dest_filepath + df_selection[x,6])

    return df_selection




In [ ]:
df_selection = random_selection(r'D:/nf/audio/',180000.00)

### Directory Structure

In [ ]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))



In [ ]:
list_files(r"D:\02_batch_2")